In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [3]:
##################################################################################

def preprocess(dirty_df):
  dirty_df = dirty_df.drop(['patient_id','cohort_type'], axis = 1)
  target_map = {u'1': 1, u'0': 0}
  dirty_df['__target__'] = dirty_df['cohort_flag'].map(str).map(target_map)
  dirty_df = dirty_df.drop(['cohort_flag'], axis = 1)
  clean_X = dirty_df.drop('__target__', axis=1)
  clean_y = np.array(dirty_df['__target__'])
  return clean_X, clean_y

def evaluateForest(ntrees, RFresults, X_train, y_train, X1_train, y1_train, X2_train, y2_train, X_test, y_test): 
  clf1 = RandomForestClassifier(n_estimators=ntrees,random_state=10000)
  clf2 = RandomForestClassifier(n_estimators=ntrees,random_state=10000)
  clf  = RandomForestClassifier(n_estimators=ntrees,random_state=10000)
  
  clf1.fit(X1_train, y1_train)
  accP1  = accuracy_score(y_test,clf1.predict(X_test))

  clf2.fit(X2_train, y2_train)
  accP2 = accuracy_score(y_test,clf2.predict(X_test))
    
  clf.fit(X_train, y_train)
  accALL = accuracy_score(y_test,clf.predict(X_test))
  
  # Merging of RF models  
  clf1.estimators_ += clf2.estimators_
  clf1.n_estimators = len(clf1.estimators_)
  accMERG = accuracy_score(y_test,clf1.predict(X_test))

  RFresults[i] = [accP1,accP2,accALL,accMERG]
  

In [21]:
##################################################################################

# Load the data
df1 = pd.read_csv('party_1.csv')
df2 = pd.read_csv('party_2.csv')

X1, y1 = preprocess(df1)
X2, y2 = preprocess(df2)

In [60]:
# party1
train_y1=y1[0:n_train]
train_X1=X1[0:n_train]
test_y1=y1[n_train:n_train+n_test]
test_x1=X1[n_train:n_train+n_test]

In [69]:
def generate_input_mpc(list_files, out_file_mpc, n_train, n_test):
    # part
    train_y1=y1[0:n_train]
    train_X1=X1[0:n_train]
    test_y1=y1[n_train:n_train+n_test]
    test_x1=X1[n_train:n_train+n_test]    
    
    train_X1.to_csv(list_files[0], header=False, sep=' ', index=False)
    y1_aux = pd.DataFrame(train_y1)
    y1_aux.to_csv(list_files[1], header=False, sep=' ', index=False)
    test_x1.to_csv(list_files[2], header=False, sep=' ', index=False)
    y2_aux = pd.DataFrame(test_y1)
    y2_aux.to_csv(list_files[3], header=False, sep=' ', index=False)
    
    
    filenames = list_files
    with open(out_file_mpc, 'w') as outfile:
        for fname in filenames:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)
                    
generate_input_mpc(['Player-Data/train_X1', 'Player-Data/train_y1', 'Player-Data/test_x1', 'Player-Data/test_y1'], 'Player-Data/Input-P0-0', n_train=700, n_test=131)

generate_input_mpc(['Player-Data/train_X2', 'Player-Data/train_y2', 'Player-Data/test_x2', 'Player-Data/test_y2'], 'Player-Data/Input-P1-0', n_train=700, n_test=131)

In [66]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report, confusion_matrix

perceptron_sklearn = Perceptron(max_iter=200, random_state=100)
perceptron_sklearn.fit(train_X1,train_y1)

resultado_predicao = perceptron_sklearn.predict(test_x1)

print('confusion_matrix\n', confusion_matrix(resultado_predicao, test_y1))
print('classification_report\n',classification_report(resultado_predicao, test_y1)) 

confusion_matrix
 [[36  6]
 [ 7 82]]
classification_report
               precision    recall  f1-score   support

           0       0.84      0.86      0.85        42
           1       0.93      0.92      0.93        89

    accuracy                           0.90       131
   macro avg       0.88      0.89      0.89       131
weighted avg       0.90      0.90      0.90       131



In [65]:
from sklearn.linear_model import LogisticRegression
regr = LogisticRegression(max_iter=200, random_state=100)
regr.fit(train_X1,train_y1)
resultado_predicao = regr.predict(test_x1)
print('confusion_matrix\n', confusion_matrix(resultado_predicao, test_y1))
print('classification_report\n',classification_report(resultado_predicao, test_y1)) 

confusion_matrix
 [[33  6]
 [10 82]]
classification_report
               precision    recall  f1-score   support

           0       0.77      0.85      0.80        39
           1       0.93      0.89      0.91        92

    accuracy                           0.88       131
   macro avg       0.85      0.87      0.86       131
weighted avg       0.88      0.88      0.88       131



In [67]:
import tensorflow as tf
from sklearn.metrics import classification_report

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid', input_shape=(1874, )))
model.summary()
model.compile(optimizer='Sgd', loss='binary_crossentropy', metrics = ['accuracy'])
epochs_hist = model.fit(train_X1, train_y1, epochs = 200) 
predict = model.predict(test_x1)
y_predict = (predict > 0.5)
print(classification_report(y_predict, test_y1)) 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 1875      
Total params: 1,875
Trainable params: 1,875
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
22/22 [==============================] - 4s 6ms/step - loss: 0.6879 - accuracy: 0.5529
Epoch 2/200
22/22 [==============================] - 0s 4ms/step - loss: 0.6224 - accuracy: 0.6471
Epoch 3/200
22/22 [==============================] - 0s 8ms/step - loss: 0.5827 - accuracy: 0.6929
Epoch 4/200
22/22 [==============================] - 0s 3ms/step - loss: 0.5489 - accuracy: 0.7443
Epoch 5/200
22/22 [==============================] - 0s 6ms/step - loss: 0.5205 - accuracy: 0.7771
Epoch 6/200
22/22 [==============================] - 0s 3ms/step - loss: 0.4953 - accuracy: 0.8114
Epoch 7/200
22/22 [==============================] -

22/22 [==============================] - 0s 7ms/step - loss: 0.1734 - accuracy: 0.9686
Epoch 77/200
22/22 [==============================] - 0s 3ms/step - loss: 0.1720 - accuracy: 0.9700
Epoch 78/200
22/22 [==============================] - 0s 5ms/step - loss: 0.1708 - accuracy: 0.9671
Epoch 79/200
22/22 [==============================] - 0s 4ms/step - loss: 0.1696 - accuracy: 0.9714
Epoch 80/200
22/22 [==============================] - 0s 6ms/step - loss: 0.1685 - accuracy: 0.9743
Epoch 81/200
22/22 [==============================] - 0s 3ms/step - loss: 0.1673 - accuracy: 0.9729
Epoch 82/200
22/22 [==============================] - 0s 7ms/step - loss: 0.1662 - accuracy: 0.9743
Epoch 83/200
22/22 [==============================] - 0s 3ms/step - loss: 0.1650 - accuracy: 0.9714
Epoch 84/200
22/22 [==============================] - 0s 4ms/step - loss: 0.1639 - accuracy: 0.9714
Epoch 85/200
22/22 [==============================] - 0s 5ms/step - loss: 0.1630 - accuracy: 0.9729
Epoch 86/200


22/22 [==============================] - 0s 3ms/step - loss: 0.1137 - accuracy: 0.9886
Epoch 156/200
22/22 [==============================] - 0s 9ms/step - loss: 0.1132 - accuracy: 0.9886
Epoch 157/200
22/22 [==============================] - 0s 3ms/step - loss: 0.1127 - accuracy: 0.9886
Epoch 158/200
22/22 [==============================] - 0s 7ms/step - loss: 0.1122 - accuracy: 0.9886
Epoch 159/200
22/22 [==============================] - 0s 3ms/step - loss: 0.1118 - accuracy: 0.9886
Epoch 160/200
22/22 [==============================] - 0s 5ms/step - loss: 0.1112 - accuracy: 0.9886
Epoch 161/200
22/22 [==============================] - 0s 4ms/step - loss: 0.1109 - accuracy: 0.9886
Epoch 162/200
22/22 [==============================] - 0s 3ms/step - loss: 0.1104 - accuracy: 0.9886
Epoch 163/200
22/22 [==============================] - 0s 5ms/step - loss: 0.1100 - accuracy: 0.9886
Epoch 164/200
22/22 [==============================] - 0s 3ms/step - loss: 0.1095 - accuracy: 0.9886
Epoc

In [3]:
# For each method, these will hold 4 accuracy results for each of 5 folds
# (1) accuracy of model trained on data from P1
# (2) accuracy of model trained on data from P2
# (3) accuracy of model trained on data from P1 and from P2
# (4) accuracy of aggregation of model (1) and (2) from above
LRresults = np.zeros((5, 4))
RF50results = np.zeros((5, 4))
RF100results = np.zeros((5, 4))
RF200results = np.zeros((5, 4))
RF400results = np.zeros((5, 4))

kf1 = KFold(n_splits=5,shuffle = True,random_state = 42)
kf2 = KFold(n_splits=5,shuffle = True,random_state = 42)


i = 0
for result1,result2 in zip(kf1.split(X1,y1),kf2.split(X2,y2)):
  print("FOLD ", i+1)
  X1_train, X1_test = X1.iloc[result1[0]], X1.iloc[result1[1]]
  y1_train, y1_test = y1[result1[0]], y1[result1[1]]
  X2_train, X2_test = X2.iloc[result2[0]], X2.iloc[result2[1]]
  y2_train, y2_test = y2[result2[0]], y2[result2[1]]

  X_train = X1_train.append(X2_train)
  y_train = np.append(y1_train,y2_train)
  X_test = X1_test.append(X2_test)
  y_test = np.append(y1_test,y2_test)


  ########## Train and test logistic regression models #################

  clf1 = LogisticRegression(solver='liblinear',random_state=10000)
  clf2 = LogisticRegression(solver='liblinear',random_state=10000)
  clf = LogisticRegression(solver='liblinear',random_state=10000)
  
  clf1.fit(X1_train, y1_train)
  accP1  = accuracy_score(y_test,clf1.predict(X_test))
  
  clf2.fit(X2_train, y2_train)
  accP2 = accuracy_score(y_test,clf2.predict(X_test))
    
  clf.fit(X_train, y_train)
  accALL = accuracy_score(y_test,clf.predict(X_test))
  
  # Merging of LR models
  clf1.coef_ = (clf1.coef_ + clf2.coef_)/2
  clf1.intercept_ = (clf1.intercept_ + clf2.intercept_)/2
  accMERG = accuracy_score(y_test,clf1.predict(X_test))  

  LRresults[i] = [accP1,accP2,accALL,accMERG]
    

  ########## Train and test RF models #################################

  ntrees=50
  RFresults=RF50results
  evaluateForest(ntrees, RFresults, X_train, y_train, X1_train, y1_train, 
                         X2_train, y2_train, X_test, y_test) 
  
  ntrees=100
  RFresults=RF100results
  evaluateForest(ntrees, RFresults, X_train, y_train, X1_train, y1_train, 
                         X2_train, y2_train, X_test, y_test) 

  ntrees=200
  RFresults=RF200results
  evaluateForest(ntrees, RFresults, X_train, y_train, X1_train, y1_train, 
                         X2_train, y2_train, X_test, y_test) 

  ntrees=400
  RFresults=RF400results
  evaluateForest(ntrees, RFresults, X_train, y_train, X1_train, y1_train, 
                         X2_train, y2_train, X_test, y_test) 


  print("==========completed")
  i = i + 1

# Printing the averages over the 5 folds
print("          P1,   P2, All, P1&P2")
np.set_printoptions(precision=2)
print("LR:     ",np.mean(LRresults, axis=0))
print("RF-50:  ",np.mean(RF50results, axis=0))
print("RF-100: ",np.mean(RF100results, axis=0))
print("RF-200: ",np.mean(RF200results, axis=0))
print("RF-400: ",np.mean(RF400results, axis=0)) 

FOLD  1
==========completed
FOLD  2
==========completed
FOLD  3
==========completed
FOLD  4
==========completed
FOLD  5
==========completed
          P1,   P2, All, P1&P2
LR:      [0.85 0.83 0.86 0.87]
RF-50:   [0.82 0.82 0.87 0.86]
RF-100:  [0.82 0.84 0.87 0.87]
RF-200:  [0.82 0.84 0.88 0.87]
RF-400:  [0.82 0.85 0.88 0.87]
